In [24]:
import re
import random
import copy

class TreeNode:  
    '''
     x(attr1)
    /\
    x(attr1=3) x(attr1=4)
    '''  
    def __init__(self, attrVal):
        self.partitionAttrVal =attrVal #标记当前node的某个attr的具体值是多少
        self.partitionAttrNum = 0  #标记在当前node下，分割的attr是哪一个
        self.children={}    #保存各种attr的node,leaf node check empty, if not l
        self.ID=None
        self.leafTag=None   #如果是leaf，就赋对应tag的值
        
def preprocess():
    print "start reading and preprocessing train data:"
    inputList=[]      #是list of list
    tagSet=set()      #统计tag有多少种，为了后面建立confusion matrix
    
#     for index, line in enumerate(open('/Users/shuofeng/Downloads/balance-scale.train', 'r').readlines()):   
    for index, line in enumerate(open('/Users/shuofeng/Downloads/led.train.new', 'r').readlines()):   
#     for index, line in enumerate(open('/Users/shuofeng/Downloads/poker.train', 'r').readlines()):   
#     for index, line in enumerate(open('/Users/shuofeng/Downloads/nursery.data.train', 'r').readlines()):   
        
        w=re.split(' |\n',line)  
        del w[-1]      #delete the new line character

        for i in range(1,len(w)):
            idx=w[i].index(':')     #找到:所在的位置
            w[i]=w[i][idx+1:]       #提取:后面的attr val
        if w[0]=="":               #需要做该检查，因为文件结尾处有可能有若干个空行
            break

        tagSet.add(w[0])          #记录有多少种tag
        
        w.append(index)        #把row number加入到entry中便于后续debug的时候知道是哪一行数据
        inputList.append(w)     #把处理好的数据加入到总的list of list中            
#     print inputList
#     print tagSet
    print "finish reading and processing train data"
    return inputList,tagSet

def testData():
    print "start read and process test data:"
    testList=[]
    
#     for index, line in enumerate(open('/Users/shuofeng/Downloads/balance-scale.test', 'r').readlines()):
    for index, line in enumerate(open('/Users/shuofeng/Downloads/led.test.new', 'r').readlines()):   
#     for index, line in enumerate(open('/Users/shuofeng/Downloads/poker.test', 'r').readlines()):   
#     for index, line in enumerate(open('/Users/shuofeng/Downloads/nursery.data.test', 'r').readlines()): 

        w=re.split(' |\n',line)   
        del w[-1]

        for i in range(1,len(w)):
            idx=w[i].index(':')     #找到:所在的位置
            w[i]=w[i][idx+1:]       #提取:后面的attr val
        if w[0]=="":               #需要做该检查，因为文件结尾处有可能有若干个空行
            break
            
        w.append(index)        #把row number加入到entry中便于后续debug的时候知道是哪一行数据
        testList.append(w)            
#     print inputList
    print "finish reading and processing test data"
    return testList   

'''
@inputList list of list 注意list of list的下标是从0开始
@rowNums list of integers
'''
def curGini(inputList,rowNums,attrNum):
    tagMap={}
    for rowNum in rowNums:
        curTag=inputList[rowNum][0]
        if curTag in tagMap.keys():
            tagMap[curTag]+=1.0
        else:
            tagMap[curTag]=1.0

    for k,v in tagMap.items():
        '''
        print "tag %s has # of rows"%k,
        print v
        '''
        
        
    sum=0.0
    for i in tagMap.keys():
        sum+=pow(tagMap[i]/len(rowNums),2)

    res=1-sum    
    return res  



'''
@inputList list of list
@rowNums list
@availAttrNums list
@root current node
'''
def nextGini(inputList,rowNums,availAttrNums,root,level,globalAvailAttrNums):
#     print
#     print "new branch"
#     print "level of tree ",
#     print level
#     print

    
    
    global nodeCount
    root.ID=nodeCount
    nodeCount+=1


    
    '''
    print "row nums: ",
    print rowNums
    print "available attr numbers: ",
    print availAttrNums
    print
    '''
    '''
    print "available attr numbers: ",
    print availAttrNums
    '''
    
    
    """    
    base case
    """
    '''
    case 1, when all the rows in rowNums list have the same label，purity=100%, stop recursion
    '''
    testSet=set()
    testMap={}
    for rowNum in rowNums:
        curList=inputList[rowNum]
        curTag=curList[0]
        if curTag not in testSet:
            testSet.add(curTag)
    if len(testSet)==1:
        root.leafTag=inputList[rowNums[0]][0]
        '''
        print "root tag:",
        print root.leafTag
        '''
        return
    '''
    case 2, when there are no available attributes, use majority voting to decide the label
    '''
    if not availAttrNums :
        for rowNum in rowNums:            
            curList=inputList[rowNum]
            curTag=curList[0]
            if curTag not in testMap:
                testMap[curTag]=1
            else:
                testMap[curTag]+=1
        max=0
        maxKey=''
        
        for k,v in testMap.items():
            if v>max:
                max=v
                maxKey=k
        root.leafTag=maxKey
        '''
        print "leaf tag:",
        print root.leafTag
        '''
#         print "here"
        '''
           print "root tag"
            print root.leafTag
        '''
        
        return 
    
    '''
    non-base case
    '''
    minSum=float("inf")
    chooseAttrNum=-1
    for attrNum in availAttrNums:#availAttrNums是一个string,记录当前branch还有哪些数据，目的是使各个branch互不影响

        curMap={}
        for rowNum in rowNums:
            curList=inputList[rowNum]#获得rowNum行的数据
           
            if curList[attrNum] in curMap.keys():#如果rowNum行数据的第n个attribute的具体值在map里，就在原来的list上追加该行的行数
                curMap[curList[attrNum]].append(rowNum)
            else:
                curMap[curList[attrNum]]=[]#如果不在，就新建一个list of list,同时记录下改行的行数
                curMap[curList[attrNum]].append(rowNum)

                
        #curMap把某一attribute的所有行放在一起，key attr, val all row numbers
        '''
        print
        print "choose attribute %d，its key:label, value:row number map:"%attrNum,
        print curMap
        '''
        
        
        
        #计算某种分割的gini,minSum记录所有分割最小gini,chooseAttrNum记录最小gini对应的attribute number
        total=float(len(rowNums))#当前branch下的总行数
        sum=0.0
        '''
        print "calculate gini score of attribute #%d for the next level"%attrNum
        '''

        for k,v in curMap.items():
            '''
            print "when attribute #%d =%s"%(attrNum,k)
            '''
            ratio=len(v)/total
            
            curTerm=ratio*curGini(inputList,v,attrNum)#关键一步
            '''
            print "gini score for this term:",
            print curTerm
            '''
            
           
            sum+=curTerm
            
        '''
        
        print "total gini score for attribute #%d :"%attrNum
        print sum
        '''
        if sum<minSum:
            minSum=sum
            chooseAttrNum=attrNum

    
    root.partitionAttrNum=chooseAttrNum#当前node记录下用了哪个attribute分割
    '''
    print
    print "attribute with min gini score:",
    print root.partitionAttrNum
    '''
    
    
    #没有gain怎么停止分割？
    
    #重新根据选中的attr建立map
    partitionMap={}
    for rowNum in rowNums:
        curList=inputList[rowNum]#获得rowNum行的数据

        if curList[chooseAttrNum] in partitionMap.keys():#如果rowNum行数据的第n个attribute的具体值在map里，就在原来的list上追加该行的行数
            partitionMap[curList[chooseAttrNum]].append(rowNum)
        else:
            partitionMap[curList[chooseAttrNum]]=[]#如果不在，就新建一个list of list,同时记录下改行的行数
            partitionMap[curList[chooseAttrNum]].append(rowNum)
    '''
    print
    print "下一层的partition map",
    print partitionMap
    '''
    
    globalAvailAttrNums=list(globalAvailAttrNums)
    '''
    print "全局available attribute"
    print globalAvailAttrNums
    print "本次选中的attribute"
    print chooseAttrNum
    '''
    
    globalAvailAttrNums.remove(chooseAttrNum)

#     if not globalAvailAttrNums or level>=maxDepth:
#         for rowNum in rowNums:            
#             curList=inputList[rowNum]
#             curTag=curList[0]
#             if curTag not in testMap:
#                 testMap[curTag]=1
#             else:
#                 testMap[curTag]+=1
#         max=0
#         maxKey=''
        
#         for k,v in testMap.items():
#             if v>max:
#                 max=v
#                 maxKey=k
#         root.leafTag=maxKey
#         '''
#         print "leaf tag:",
#         print root.leafTag
#         '''
        
#         '''
#            print "root tag"
#             print root.leafTag
#         '''
        
#         return    
    newAvailAttrNums=[]
    if globalAvailAttrNums:
        
        attrPerNode=int((len(inputList[0])-2)**0.5)
        for i in range(attrPerNode):
            r = random.randint(0, len(globalAvailAttrNums)-1)
            newAvailAttrNums.append(globalAvailAttrNums[r])
    #     newAvailAttrNums=[1,2,3,4,5,6,7]

    #     for i in visited:
    #         if i in newAvailAttrNums:
    #             newAvailAttrNums.remove(i)
    #     newVisited=copy.deepcopy(visited)
    #     for i in newAvailAttrNums:
    #         newVisited.add(i)

 
    
    for k,v in partitionMap.items():
        
        newRowNums=v
        newRoot=TreeNode(k)  #生成child 节点
        root.children[k]=newRoot  #把child 节点连接到当前节点上

        nextGini(inputList,newRowNums,newAvailAttrNums,newRoot,level+1,globalAvailAttrNums) #下一层循环
    
        

if __name__ == "__main__":

    
    nodeCount=0
    inputList,tagSet=preprocess()#list of list
    '''
    build confusion matrix
    '''
    
    outPutListList=[]
    for i in range(len(tagSet)):
        tmpList=[0 for j in range(len(tagSet))]
        outPutListList.append(tmpList)
 
    
    '''
        参数调节
    '''
    nodeList=[]
    print "总的数据量:",        
    print len(inputList)
    
    numTreefactor=3
    print "树的个数:",
#     numTree=int(0.5*numTreefactor*len(inputList)**0.5)
    numTree=260
    print numTree
    
    print "树的最大深度"
    maxDepth=int(0.09*(len(inputList)**0.5))
    print maxDepth
    
    print "每个树用的点的个数"
#     dataPerTree=int(0.68*len(inputList)**0.5)
#     dataPerTree=int(60*len(inputList)**0.5)
    dataPerTree=int(1.5*len(inputList))
#     dataPerTree=int(len(inputList))
    print dataPerTree
    
    
    
    print "每个node用的attribute个数"
    attrPerNode=int((len(inputList[0])-2)**0.5)
    print attrPerNode
    
    globalAvailAttrNums=[i for i in range(1,len(inputList[0])-1)]
    print globalAvailAttrNums

    
    for i in range(numTree):#树的个数
        
        rowNums=[]
        for i in range(dataPerTree):#每个树用的点的个数
            r = random.randint(0, len(inputList)-1)
            rowNums.append(r)
                 
        availAttrNums=[]
        for i in range(attrPerNode):#每个node用的attribute个数
            r = random.randint(1, len(inputList[0])-2)
            availAttrNums.append(r)
#         availAttrNums=[1,2,3,4,5,6,7]
    
        level=0
        root=TreeNode(-1)
        nodeList.append(root)
        visited=set()
        nextGini(inputList,rowNums,availAttrNums,root,level,globalAvailAttrNums)

    
    '''
        test step
    '''
    print "test phase"
    trueCount=0
    falseCount=0
    testList=testData()#list of list
    for test in testList:
        voteMap={}
        for node in nodeList:
            stack=[]
            stack.append(root)  
            while stack:
                cur=stack.pop()
    #             print "current ID:"
    #             print cur.ID    

                testAttrValue=str(test[cur.partitionAttrNum])
    #             print "test attr value"
    #             print testAttrValue

                if not cur.children:#如果当前的label不在数据集了，就算作第一个branch
                    '''
                    print "leaf tag:",
                    print cur.leafTag

                    '''
                    if cur.leafTag in voteMap.keys():
                        voteMap[cur.leafTag]+=1
                    else:
                        voteMap[cur.leafTag]=1


                    break
                if testAttrValue in cur.children.keys():
                    newNode=cur.children[testAttrValue]
                else:
                    newNode=cur.children[cur.children.keys()[0]]
                stack.append(newNode)

        majorityTag=None
        max=-1
        for k,v in voteMap.items():
            if v>max:
                majority=k
                    
        outPutListList[int(test[0])-1][int(majority)-1]+=1

    #output confusion matrix
    print "confusion matrix:"
    for i in outPutListList:
        print i
        
    '''
        evaluation
    '''
    numeritor=0
    denominator=0
    for i in range(len(outPutListList)):
        numeritor+=outPutListList[i][i]

    for i in range(len(outPutListList)):
        for j in range(len(outPutListList)):
            denominator+=outPutListList[i][j]
    print "accuracy is:",
    accuracy=float(numeritor)/float(denominator)
    print accuracy

    with open("RFtest.txt", "a") as myfile:
        myfile.write(str(numTreefactor)+" "+str(numTree)+" "+str(dataPerTree)+" "+str(attrPerNode)+" "+str(accuracy)+ " "+str(maxDepth))
        myfile.write("\n")

start reading and preprocessing train data:
finish reading and processing train data
总的数据量: 2087
树的个数: 260
树的最大深度
4
每个树用的点的个数
3130
每个node用的attribute个数
2
[1, 2, 3, 4, 5, 6, 7]
test phase
start read and process test data:
finish reading and processing test data
confusion matrix:
[266, 85]
[77, 706]
accuracy is: 0.857142857143


In [2]:
import copy
x=set()
x.add(1)
x.add(2)
y=copy.deepcopy(x)
y.add(3)

print x
print y


l=[]
if l:
    print "haha"

set([1, 2])
set([1, 2, 3])
